## ABCs Abstract Base Classes

ABCs are meant to encapsulate very general concepts, abstractions, introduced by a framework - things like "a sequence" and "an exact number".

Why ABCs:
- By inheriting from ABCs we make a clear declaration of intent and are forced to implement all of the relevant methods of a given protocol.
- If we check isinstance() we are mostly interested whether the object follows a certain protocol (not whether it is an object of a certain type) and as such it is better to tests against ABCs.
- Checking against ABC with isinstance is especially useful if you must enforce an API contract: "Dude, you have to implement this if you want to call me".

However, excessive type checking might be a sign of poor design. You should be leveraging polymorphism, designing your classes so that the interpreter dispatches calls to proper methods instead of hard coding choices based on types. For example, if you built a class that requires `list` argument, instead of checking you can immediately build a `list` from it, this way you can accept any iterable.

In [1]:
# Subclassing an ABC
import collections

class FrenchDeck2(collections.MutableSequence):
    pass

In [2]:
deck = FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract methods __delitem__, __getitem__, __len__, __setitem__, insert

This is exactly how ABCs enforce a protocol. Note that not all methods of MutableSequence are abstract. FrenchDeck inherits ready-to-use concrete methods `__contains__`, `__iter__` ... from Sequence. However, you might still with to override them with more efficient impleentations.

In [3]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        # Subclassing ABCs forced us to implement del item...
        del self._cards[position]
    
    def insert(self, position, value):
        self._cards.insert(position, value)

In [4]:
deck = FrenchDeck2()

Python does not check for implementation of the abstract methods at import time, but only at runtime when we actually instantiate FrenchDeck2.

## ABCs in stdlib

ABC are implemented in `_collections_abc.py` there are roughly 21 as of 3.6. However, they remain vsible in the collections module and you can use them importing `collections` instead of `collections.abc`

### Iterable, Container and Sized
Every collection should inherit from these or at least implement compatible protocols. Iterable supports iteration with `__iter__`, Container supports the `in` operator with `__contains__` and Sized supports `len()` with `__len__`.

### Sequence, Mapping and Set
Main immutable collections and each has a mutable subclass.

### MappingView
The objects returned from the mapping methods `.items()`, `.keys()` and `.values()`

### Callable and Hashable
Their main use is to check whether an object is hashable. You can check whether an object is callable by calling built-in `callable()`.

### Iterator
The all mighty subclass of Iterable!

## Numbers

numbers package defines the linear hierarchy of numerical ABCs:
- `Number`
- `Complex`
- `Real`
- `Rational`
- `Integral`

You check `isinstance(x, numbers.Integral)` to accept `int`, `bool` and other integer types. 